<a href="https://colab.research.google.com/github/berholz/GasTurbineEngineClassifier/blob/main/%D1%82%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D1%80%D0%B5%D0%BA%D0%BB%D0%B0%D0%BC%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
#data=pd.read_csv('stager1.csv', delimiter=';') 
from google.colab import drive
drive.mount('/content/drive')
path = "drive/My Drive/stager.csv"
data= pd.read_csv(path, delimiter=';')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Посмотрим на таблицу

In [7]:
data.head()

,Client,client_id,Month,Category,Device,Place,BannerType,QueryType (search only),TargetingType,Shows,Clicks,Cost_rub_wo_NDS,Сonversions
0,Бета,234679,01.07.2020,ВКЛАДЫ,Desktop,Networks,cpc_video,none,Phrase,0,0,0.000000,0
1,Гамма,458303,01.04.2019,ДЕБЕТОВЫЕ КАРТЫ,Desktop,Networks,cpc_video,none,Phrase,76722,2002,19467.482340,3
2,Гамма,458303,01.04.2019,ДЕБЕТОВЫЕ КАРТЫ,Desktop,Networks,cpc_video,none,Retargeting,9260,116,2590.924703,0
3,Гамма,458303,01.06.2019,ДЕБЕТОВЫЕ КАРТЫ,Desktop,Networks,cpc_video,none,Retargeting,0,0,0.000000,0
4,Гамма,458303,01.04.2019,ДЕБЕТОВЫЕ КАРТЫ,Mobile,Networks,cpc_video,none,Phrase,27140,825,4181.504949,0




![image.png](attachment:image.png)

In [8]:
data['Month']= pd.to_datetime(data['Month'], format='%d.%m.%Y')

In [9]:
data['Client'].unique()

array(['Бета', 'Гамма', 'Псилон', 'Сигма', 'Дельта', 'Микрон', 'Альфа',
       'Зета', 'Мета', 'Омега', 'Лямбда'], dtype=object)

Анализируемая категория-потребительское кредитование

In [10]:
data1=data[data['Category']=='ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ']

In [11]:
data1.head()

,Client,client_id,Month,Category,Device,Place,BannerType,QueryType (search only),TargetingType,Shows,Clicks,Cost_rub_wo_NDS,Сonversions
28,Сигма,357953,2019-04-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,cpc_video,none,Phrase,1114,17,398.412941,0
29,Сигма,357953,2019-05-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,cpc_video,none,Phrase,6160,74,1447.873576,0
30,Сигма,357953,2019-09-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,cpc_video,none,Retargeting,0,0,0.000000,0
31,Сигма,357953,2019-10-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,cpc_video,none,Retargeting,0,0,0.000000,0
32,Сигма,357953,2019-11-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,cpc_video,none,Retargeting,0,0,0.000000,0


In [12]:
data['Client'].unique()

array(['Бета', 'Гамма', 'Псилон', 'Сигма', 'Дельта', 'Микрон', 'Альфа',
       'Зета', 'Мета', 'Омега', 'Лямбда'], dtype=object)

In [13]:
names=['Бета', 'Гамма', 'Псилон', 'Сигма', 'Дельта', 'Микрон', 'Альфа',
       'Зета', 'Мета', 'Омега', 'Лямбда']

Анализируемый промежуток времени 01.01.2020-31.08.2020

In [14]:
data_to_analize =data1.loc[(data['Month']>'2020-01-01') &  (data['Month']<'2020-08-31')]

In [15]:
len(data_to_analize)

1027

In [16]:
data_to_analize['Client'].unique()

array(['Бета', 'Гамма', 'Сигма', 'Альфа', 'Омега', 'Лямбда', 'Зета',
       'Мета', 'Псилон'], dtype=object)

In [17]:
data_to_analize[data_to_analize['Client']== 'Омега'].head(15)

,Client,client_id,Month,Category,Device,Place,BannerType,QueryType (search only),TargetingType,Shows,Clicks,Cost_rub_wo_NDS,Сonversions
1137,Омега,333348,2020-08-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,image_ad,none,Retargeting,0,0,0.000000e+00,0
8923,Омега,333348,2020-02-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Search,text,Brand,Phrase,177791,23798,1.138670e+06,908
8924,Омега,333348,2020-02-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Search,text,Competitor,Phrase,14004,655,4.780165e+04,24
8925,Омега,333348,2020-02-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Search,text,Other,Phrase,157544,16652,1.383995e+06,992
8926,Омега,333348,2020-03-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,text,none,Phrase,0,0,0.000000e+00,0
8927,Омега,333348,2020-03-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Search,text,Brand,Phrase,237483,32671,1.392743e+06,2326
8928,Омега,333348,2020-03-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Search,text,Competitor,Phrase,15008,810,5.511982e+04,38
8929,Омега,333348,2020-03-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Search,text,Other,Phrase,194465,18369,1.526398e+06,1925
8930,Омега,333348,2020-04-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,text,none,Retargeting,0,0,0.000000e+00,0
8931,Омега,333348,2020-04-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Search,text,Brand,Phrase,277293,34938,1.450623e+06,3239


Рекламный бюджет компании

In [18]:
data_omega=data_to_analize[data_to_analize['Client']=='Омега']

In [19]:
name_tool=['Device', 'Place','BannerType','TargetingType']
titles=['Распределение расходов  Омеги по типам устройств', 'Распределение расходов Омеги по месту размещения', 'Распределение расходов Омеги по типу баннера',
       'Распределение расходов Омеги по типам таргетинга']

i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS'], aggfunc='sum') 
    #data_omega_gr['CR']=data_omega_gr['Сonversions']*100/data_omega_gr['Shows']
    print(data_omega_gr)
    #data_other_gr=data_other.pivot_table(index=nm,  values=['Shows','Сonversions'], aggfunc='sum') 
    #data_other_gr['CR']=data_other_gr['Сonversions']*100/data_other_gr['Shows']
    fig = go.Figure()
    fig.add_trace(go.Pie(name='Omega',values=data_omega_gr['Cost_rub_wo_NDS'], labels=data_omega_gr.index))
    #fig.add_trace(go.Pie(name='Другие',values=data_other_gr['CR'], labels=data_other_gr.index))
    fig.update_layout(title=titles[i],
                     )
    fig.show()
    i+=1                 

         Cost_rub_wo_NDS
Device                  
Desktop     1.293438e+07
Mobile      1.406173e+07


          Cost_rub_wo_NDS
Place                    
Networks     0.000000e+00
Search       2.699611e+07


            Cost_rub_wo_NDS
BannerType                 
cpm_banner     0.000000e+00
image_ad       0.000000e+00
text           2.699611e+07


               Cost_rub_wo_NDS
TargetingType                 
Autotargeting     0.000000e+00
Phrase            2.699611e+07
Retargeting       0.000000e+00


In [20]:
data_cr=data_to_analize.pivot_table(index='Client',  values=['Cost_rub_wo_NDS'], aggfunc='sum') 


data_cr.sort_values(by = 'Cost_rub_wo_NDS',ascending = False)


,Cost_rub_wo_NDS
Client,
Бета,1.114799e+08
Лямбда,5.937816e+07
Сигма,5.562250e+07
Псилон,3.407521e+07
Альфа,3.008922e+07
Зета,2.893528e+07
Омега,2.699611e+07
Гамма,2.657636e+07
Мета,2.044487e+07


1 . Коэффициент конверсии (сonversion rate, CR) — это отношение количества целевых визитов к общему количеству посещений сайта или приложения. CR выражается в процентах.



In [21]:
table_score=pd.DataFrame( index=None, columns=['метрика', 'Омега','Бета', 'Гамма', 'Псилон', 'Сигма', 'Дельта', 'Микрон', 'Альфа',
       'Зета', 'Мета', 'Лямбда'])
table_score.loc[0,'метрика']='сonversion rate, CR'


In [22]:
data_cr=data_to_analize.pivot_table(index='Client',  values=['Shows','Сonversions'], aggfunc='sum') 

data_cr['CR']=data_cr['Сonversions']*100/data_cr['Shows']
data_cr=data_cr.sort_values(by = 'CR',ascending = True)
data_cr.head(10)

,Shows,Сonversions,CR
Client,,,
Бета,787595027,175098,0.022232
Гамма,129255292,100982,0.078126
Псилон,59200491,47336,0.079959
Лямбда,262457846,221585,0.084427
Сигма,247914686,211496,0.085310
Мета,91627496,130087,0.141974
Альфа,30952161,49896,0.161204
Зета,4423384,17357,0.392392
Омега,3903778,66357,1.699815


In [23]:
data_cr.index

Index(['Бета', 'Гамма', 'Псилон', 'Лямбда', 'Сигма', 'Мета', 'Альфа', 'Зета',
       'Омега'],
      dtype='object', name='Client')

In [24]:
fig = go.Figure()
fig.add_trace(go.Bar(x = data_cr.index, y = data_cr['CR']))
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Сто",
                  xaxis_title="Название фирмы",
                  yaxis_title="коэффицинт конверрсии в %",
                  margin=dict(l=0, r=0, t=50, b=0))
fig.show()

In [25]:
fig = go.Figure()
fig.add_trace(go.Bar(x = data_cr.index, y = data_cr['Сonversions']))
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Количество конверсий за выбранный период",
                  xaxis_title="Название фирмы",
                  yaxis_title="Сумма конверрсии ",
                  margin=dict(l=0, r=0, t=50, b=0))
fig.show()

In [26]:
data_omega=data_to_analize[data_to_analize['Client']=='Омега']
data_omega['Place'].value_counts()

Search      45
Networks    31
Name: Place, dtype: int64

In [27]:
data_other=data_to_analize[data_to_analize['Client']!='Омега']


In [28]:
name_tool=['Device', 'Place','BannerType','TargetingType']
titles=['Доля, вносимая в CR Омеги по типам устройств', 'Доля, вносимая в CR Омеги по месту размещения', 'Доля, вносимая в CR Омеги по типу баннера',
       'Доля, вносимая в CR Омеги по типам таргетинга']

i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Shows','Сonversions'], aggfunc='sum') 
    data_omega_gr['CR']=data_omega_gr['Сonversions']*100/data_omega_gr['Shows']
    print(data_omega_gr)
    #data_other_gr=data_other.pivot_table(index=nm,  values=['Shows','Сonversions'], aggfunc='sum') 
    #data_other_gr['CR']=data_other_gr['Сonversions']*100/data_other_gr['Shows']
    fig = go.Figure()
    fig.add_trace(go.Pie(name='Omega',values=data_omega_gr['CR'], labels=data_omega_gr.index))
    #fig.add_trace(go.Pie(name='Другие',values=data_other_gr['CR'], labels=data_other_gr.index))
    fig.update_layout(title=titles[i],
                     )
    fig.show()
    i+=1                 

           Shows  Сonversions        CR
Device                                 
Desktop  1882453        20873  1.108819
Mobile   2021325        45484  2.250207


            Shows  Сonversions        CR
Place                                   
Networks        0            1       inf
Search    3903778        66356  1.699789


              Shows  Сonversions        CR
BannerType                                
cpm_banner        0            0       NaN
image_ad          0            0       NaN
text        3903778        66357  1.699815


                 Shows  Сonversions        CR
TargetingType                                
Autotargeting        0            0       NaN
Phrase         3903778        66356  1.699789
Retargeting          0            1       inf


In [29]:
name_tool=['Device', 'Place','BannerType','TargetingType']


titles=['Доля, вносимая в CR других компаний по типам устройств', 'Доля, вносимая в CR других компаний по месту размещения', 'Доля, вносимая в CR других компаний по типу баннера',
       'Доля, вносимая в CR по типам таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_other.pivot_table(index=nm,  values=['Shows','Сonversions'], aggfunc='sum') 
    data_omega_gr['CR']=data_omega_gr['Сonversions']*100/data_omega_gr['Shows']
    print(data_omega_gr)
    fig = go.Figure()
    fig.add_trace(go.Pie(values=data_omega_gr['CR'], labels=data_omega_gr.index))
    fig.update_layout(title=titles[i],
                     )
    fig.show()
    i+=1       

              Shows  Сonversions        CR
Device                                    
Desktop  1093042950       323696  0.029614
Mobile    520383433       630141  0.121092


               Shows  Сonversions        CR
Place                                      
Networks  1565969236       132748  0.008477
Search      47457147       821089  1.730169


                  Shows  Сonversions        CR
BannerType                                    
cpc_video       2351101            3  0.000128
cpm_banner     19521749          331  0.001696
image_ad       32422219         1097  0.003383
mcbanner        2052403         4170  0.203176
performance           0            0       NaN
text         1557078911       948236  0.060898


                    Shows  Сonversions        CR
TargetingType                                   
Autotargeting      618761         2700  0.436356
Phrase         1527211847       939935  0.061546
Retargeting      85595775        11202  0.013087


2. Стоимость обращения (cost per action, CPA) показывает, сколько стоит целевой визит посетителя. CPA — отношение рекламного бюджета к количеству обращений.

In [30]:
data_cr=data_to_analize.pivot_table(index='Client',  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 

data_cr['CPA']=data_cr['Cost_rub_wo_NDS']/data_cr['Сonversions']
data_sr=data_cr.sort_values(by='CPA')
data_sr.head(10)

,Cost_rub_wo_NDS,Сonversions,CPA
Client,,,
Мета,2.044487e+07,130087,157.163086
Сигма,5.562250e+07,211496,262.995501
Гамма,2.657636e+07,100982,263.179153
Лямбда,5.937816e+07,221585,267.970124
Омега,2.699611e+07,66357,406.831328
Альфа,3.008922e+07,49896,603.038655
Бета,1.114799e+08,175098,636.671395
Псилон,3.407521e+07,47336,719.858232
Зета,2.893528e+07,17357,1667.066715


In [31]:
fig = go.Figure()
fig.add_trace(go.Bar(x = data_cr.index, y = data_cr['CPA']))
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="CPA за выбранный период",
                  xaxis_title="Название фирмы",
                  yaxis_title="CPA",
                  margin=dict(l=0, r=0, t=50, b=0))
fig.show()

In [32]:
name_tool=['Device', 'Place']#,'BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения']#, 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    data_other_gr=data_other.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [33]:
name_tool=['BannerType','TargetingType']

titles=['CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    data_other_gr=data_other.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [34]:
data_other_zeta=data_to_analize[data_to_analize['Client']=='Зета']
data_other_no_zeta=data_to_analize[(data_to_analize['Client']!='Зета')&(data_to_analize['Client']!='Омега') ]

In [35]:
name_tool=['Device', 'Place']#,'BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения']#, 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Zeta',x = data_zeta_gr.index, y = data_zeta_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [44]:
name_tool=['Device', 'Place']#,'BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения']#, 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Zeta',x = data_zeta_gr.index, y = data_zeta_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [ ]:
name_tool=['BannerType','TargetingType']

titles=['CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Zeta',x = data_zeta_gr.index, y = data_zeta_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [43]:
name_tool=['Device', 'Place']#,'BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения']#, 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    print(titles[i])
    i+=1
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    print('omega')
    print(data_omega_gr)
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    print('Zeta')
    print(data_zeta_gr)
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    print('Other')
    print(data_other_gr)

CPA по типам устройств
omega
         Cost_rub_wo_NDS  Сonversions         CPA
Device                                           
Desktop     1.293438e+07        20873  619.670191
Mobile      1.406173e+07        45484  309.157738
Zeta
         Cost_rub_wo_NDS  Сonversions          CPA
Device                                            
Desktop     1.571990e+07        15972   984.216223
Mobile      1.321538e+07         1385  9541.787335
Other
         Cost_rub_wo_NDS  Сonversions         CPA
Device                                           
Desktop     6.788159e+07       221425  306.566959
Mobile      1.242295e+08       492621  252.180714
CPA по месту размещения
omega
          Cost_rub_wo_NDS  Сonversions         CPA
Place                                             
Networks     0.000000e+00            1    0.000000
Search       2.699611e+07        66356  406.837459
Zeta
          Cost_rub_wo_NDS  Сonversions          CPA
Place                                              
Networks     

In [52]:
data_other_psy=data_to_analize[data_to_analize['Client']=='Псилон']
data_other_no_zeta=data_to_analize[(data_to_analize['Client']!='Зета')&(data_to_analize['Client']!='Омега') & (data_to_analize['Client']!='Псилон')& (data_to_analize['Client']!='Бета') ]
data_beta=data_to_analize[data_to_analize['Client']=='Бета']
name_tool=['BannerType','TargetingType']

titles=['CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    
    data_psy_gr=data_other_psy.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_psy_gr['CPA']=data_psy_gr['Cost_rub_wo_NDS']/data_psy_gr['Сonversions']
    
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']

    data_beta_gr=data_beta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_beta_gr['CPA']=data_beta_gr['Cost_rub_wo_NDS']/data_beta_gr['Сonversions']

    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Zeta',x = data_zeta_gr.index, y = data_zeta_gr['CPA']))
    fig.add_trace(go.Bar(name='Psylon',x = data_psy_gr.index, y = data_psy_gr['CPA']))
    fig.add_trace(go.Bar(name='Beta', x = data_beta_gr.index, y = data_beta_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [50]:
name_tool=['Device', 'Place']#,'BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения']#, 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    
    data_psy_gr=data_other_psy.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_psy_gr['CPA']=data_psy_gr['Cost_rub_wo_NDS']/data_psy_gr['Сonversions']
    
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']

    data_beta_gr=data_beta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_beta_gr['CPA']=data_beta_gr['Cost_rub_wo_NDS']/data_beta_gr['Сonversions']

    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Zeta',x = data_zeta_gr.index, y = data_zeta_gr['CPA']))
    fig.add_trace(go.Bar(name='Psylon',x = data_psy_gr.index, y = data_psy_gr['CPA']))
    #fig.add_trace(go.Bar(name='Beta', x = data_beta_gr.index, y = data_beta_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [40]:
data_other_no_zeta[data_other_no_zeta['BannerType']=='cpc_video']

,Client,client_id,Month,Category,Device,Place,BannerType,QueryType (search only),TargetingType,Shows,Clicks,Cost_rub_wo_NDS,Сonversions
35,Бета,234679,2020-07-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,cpc_video,none,Phrase,1845674,8005,70685.545930,0
36,Бета,234679,2020-07-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Desktop,Networks,cpc_video,none,Retargeting,315901,2042,18573.896060,0
37,Бета,234679,2020-07-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Mobile,Networks,cpc_video,none,Phrase,34037,465,2974.504195,0
38,Бета,234679,2020-07-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Mobile,Networks,cpc_video,none,Retargeting,155489,1251,9388.322008,3


In [41]:
data_other_psy=data_to_analize[data_to_analize['Client']=='Псилон']
data_other_no_zeta=data_to_analize[(data_to_analize['Client']!='Зета')&(data_to_analize['Client']!='Омега') & (data_to_analize['Client']!='Псилон') & (data_to_analize['Client']!='Бета') ]

name_tool=['BannerType','TargetingType']

titles=['CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    
    data_psy_gr=data_other_psy.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_psy_gr['CPA']=data_psy_gr['Cost_rub_wo_NDS']/data_psy_gr['Сonversions']
    
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Zeta',x = data_zeta_gr.index, y = data_zeta_gr['CPA']))
    fig.add_trace(go.Bar(name='Psylon',x = data_psy_gr.index, y = data_psy_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [51]:
name_tool=['Device', 'Place']#,'BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения']#, 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    
    data_psy_gr=data_other_psy.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_psy_gr['CPA']=data_psy_gr['Cost_rub_wo_NDS']/data_psy_gr['Сonversions']
    
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Omega',x = data_omega_gr.index, y = data_omega_gr['CPA']))
    fig.add_trace(go.Bar(name='Zeta',x = data_zeta_gr.index, y = data_zeta_gr['CPA']))
    fig.add_trace(go.Bar(name='Psylon',x = data_psy_gr.index, y = data_psy_gr['CPA']))
    fig.add_trace(go.Bar(name='Другие', x = data_other_gr.index, y = data_other_gr['CPA']))
    fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.1, xanchor="center"),
                      title=titles[i],
                      xaxis_title="Тип",
                      yaxis_title="CPA",
                      margin=dict(l=0, r=0, t=50, b=0))
    fig.show()
    i+=1 

In [54]:
name_tool=['Device', 'Place','BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения', 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    print(titles[i])
    i+=1
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    print('omega')
    print(data_omega_gr)    
    data_zeta_gr=data_other_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_zeta_gr['CPA']=data_zeta_gr['Cost_rub_wo_NDS']/data_zeta_gr['Сonversions']
    print('zeta')
    print(data_zeta_gr)
    data_psy_gr=data_other_psy.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_psy_gr['CPA']=data_psy_gr['Cost_rub_wo_NDS']/data_psy_gr['Сonversions']
    print('psy')
    print(data_zeta_gr)
    data_beta_gr=data_beta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_beta_gr['CPA']=data_beta_gr['Cost_rub_wo_NDS']/data_beta_gr['Сonversions']
    print('beta')
    print(data_beta_gr)
    data_other_gr=data_other_no_zeta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_other_gr['CPA']=data_other_gr['Cost_rub_wo_NDS']/data_other_gr['Сonversions']
    print('other')
    print(data_other_gr)
    data_beta_gr=data_beta.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_beta_gr['CPA']=data_beta_gr['Cost_rub_wo_NDS']/data_beta_gr['Сonversions']

CPA по типам устройств
omega
         Cost_rub_wo_NDS  Сonversions         CPA
Device                                           
Desktop     1.293438e+07        20873  619.670191
Mobile      1.406173e+07        45484  309.157738
zeta
         Cost_rub_wo_NDS  Сonversions          CPA
Device                                            
Desktop     1.571990e+07        15972   984.216223
Mobile      1.321538e+07         1385  9541.787335
psy
         Cost_rub_wo_NDS  Сonversions          CPA
Device                                            
Desktop     1.571990e+07        15972   984.216223
Mobile      1.321538e+07         1385  9541.787335
beta
         Cost_rub_wo_NDS  Сonversions         CPA
Device                                           
Desktop     4.445416e+07        72492  613.228450
Mobile      6.702573e+07       102606  653.234032
other
         Cost_rub_wo_NDS  Сonversions         CPA
Device                                           
Desktop     6.788159e+07       221425  306.

In [55]:
name_tool=['Device', 'Place','BannerType','TargetingType']

titles=['CPA по типам устройств', 'CPA по месту размещения', 'CPA  по типу баннера', 'CPA по типу таргетинга']
i=0
for nm in name_tool:
    print(titles[i])
    i+=1
    data_omega_gr=data_omega.pivot_table(index=nm,  values=['Cost_rub_wo_NDS','Сonversions'], aggfunc='sum') 
    data_omega_gr['CPA']=data_omega_gr['Cost_rub_wo_NDS']/data_omega_gr['Сonversions']
    print('omega')
    print(data_omega_gr)

CPA по типам устройств
omega
         Cost_rub_wo_NDS  Сonversions         CPA
Device                                           
Desktop     1.293438e+07        20873  619.670191
Mobile      1.406173e+07        45484  309.157738
CPA по месту размещения
omega
          Cost_rub_wo_NDS  Сonversions         CPA
Place                                             
Networks     0.000000e+00            1    0.000000
Search       2.699611e+07        66356  406.837459
CPA  по типу баннера
omega
            Cost_rub_wo_NDS  Сonversions         CPA
BannerType                                          
cpm_banner     0.000000e+00            0         NaN
image_ad       0.000000e+00            0         NaN
text           2.699611e+07        66357  406.831328
CPA по типу таргетинга
omega
               Cost_rub_wo_NDS  Сonversions         CPA
TargetingType                                          
Autotargeting     0.000000e+00            0         NaN
Phrase            2.699611e+07        66356  406